In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
!pip install yfinance
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Use Webscraping to Extract List of S&P 500 Companies </li>
        <li>Use Webscraping to Extract List of S&P 500 Components </li>
    </ul>
<p>
</div>

<hr>


### Use Webscraping to Extract List of S&P 500 Companies

Use the `requests` library to download the webpage Save the text of the response as a variable named `html_data`.

In [ ]:
html_data=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text

Parse the html data using `beautiful_soup`.

In [ ]:
beautiful_soup=BeautifulSoup(html_data, "html.parser")

Using `BeautifulSoup` or the `read_html` function extract the table

In [ ]:
tables = beautiful_soup.find_all('table')
len(tables)

2

In [ ]:
S_P_500_companies = pd.DataFrame(columns=["Symbol","Security","Sector","Sub-Industry","Headquarters Location","Date first added","CIK","Founded"])

for row in tables[0].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Symbol = col[0].text.strip().replace('\n','')
        Security = col[1].text.strip().replace('\n','')
        Sector = col[3].text.strip().replace('\n','')
        Sub_Industry = col[4].text.strip().replace('\n','')
        Headquarters_Location = col[5].text.strip().replace('\n','')
        Date_first = col[6].text.strip().replace('\n','')
        CIK = col[7].text.strip().replace('\n','')
        Founded = col[8].text.strip().replace('\n','')
        S_P_500_companies = S_P_500_companies.append({"Symbol":Symbol, "Security":Security, "Sector":Sector, "Sub-Industry":Sub_Industry,
        "Headquarters Location":Headquarters_Location,"Date first added":Date_first,"CIK":CIK,"Founded":Founded}, ignore_index=True)

In [ ]:
S_P_500_companies.head()

,Symbol,Security,Sector,Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981


In [ ]:
Symbols=S_P_500_companies['Symbol'].to_list()
len(Symbols)

503

### Use Webscraping to Extract List of S&P 500 Components

In [ ]:
Symbols_data= {} # empty dictionary
for Symbol in Symbols:
    ticker_object = yf.Ticker(Symbol)

    #convert info() output from dictionary to dataframe
    temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
    temp.reset_index(inplace=True)
    temp.columns = ["Attribute", "Recent"]
    temp['Symbol']=Symbol
    # add (ticker, dataframe) to main dictionary
    Symbols_data[Symbol] = temp
combined_data = pd.concat(Symbols_data)
combined_data

Attribute  \
MMM 0                    zip   
    1                 sector   
    2      fullTimeEmployees   
    3    longBusinessSummary   
    4                   city   
...                      ...   
ZTS 148              bidSize   
    149              dayHigh   
    150   regularMarketPrice   
    151       preMarketPrice   
    152             logo_url   

                                                    Recent Symbol  
MMM 0                                           55144-1000    MMM  
    1                                          Industrials    MMM  
    2                                                95000    MMM  
    3    3M Company operates as a diversified technolog...    MMM  
    4                                           Saint Paul    MMM  
...                                                    ...    ...  
ZTS 148                                                800    ZTS  
    149                                            173.545    ZTS  
    150                                             171.89    ZTS  
    151                                               None    ZTS  
    152               https://logo.clearbit.com/zoetis.com    ZTS  

[76667 rows x 3 columns]

In [ ]:
s=combined_data.pivot(index='Symbol', columns='Attribute', values='Recent')
s.reset_index(inplace=True)
s.columns.name=None
SP500Components=s.set_index('Symbol')
SP500Components.head()

,52WeekChange,SandP52WeekChange,address1,address2,algorithm,annualHoldingsTurnover,annualReportExpenseRatio,ask,askSize,averageDailyVolume10Day,...,underlyingExchangeSymbol,underlyingSymbol,uuid,volume,volume24Hr,volumeAllCurrencies,website,yield,ytdReturn,zip
Symbol,,,,,,,,,,,,,,,,,,,,,
A,-0.201922,-0.130266,5301 Stevens Creek Boulevard,NaN,None,None,None,119.6,800,1978850,...,NaN,NaN,NaN,1497266,None,None,https://www.agilent.com,None,None,95051
AAL,None,None,1 Skyview Drive,NaN,None,None,None,12.77,36100,38970440,...,None,None,b73d98bf-8d5e-382f-9d49-b84d8f52f152,37521383,None,None,https://www.aa.com,None,None,76155
AAP,-0.179863,-0.130266,4200 Six Forks Road,NaN,None,None,None,0,800,848670,...,NaN,NaN,NaN,1000032,None,None,https://www.advanceautoparts.com,None,None,27604
AAPL,None,None,One Apple Park Way,NaN,None,None,None,0,1000,86053440,...,None,None,8b10e4ae-9eeb-3684-921a-9ab27e4d87aa,98772673,None,None,https://www.apple.com,None,None,95014
ABBV,0.32986,-0.130266,1 North Waukegan Road,NaN,None,None,None,153.54,900,7341100,...,NaN,NaN,NaN,7202524,None,None,https://www.abbvie.com,None,None,60064-6400


In [ ]:
import os
os.chdir('/kaggle/working')
SP500Components.to_csv('/kaggle/working/S&P500_Components.csv')
S_P_500_companies.to_csv('/kaggle/working/S&P500_Companies.csv')
